## 1. Встановлення залежностей

In [47]:
# Завантажуємо необхідні бібліотеки
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark

## 2. Налаштування середовища для роботи зі Spark

In [48]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

# Перевіряємо, чи існує SparkContext, якщо так - використовуємо його
try:
    sc = SparkContext.getOrCreate()
except ValueError:
    conf = SparkConf().setAppName("InvertedIndex").setMaster("local[*]")
    sc = SparkContext(conf=conf)

spark = SparkSession(sc)

## 3. Розпаковка архіву з даними

In [49]:
import tarfile

tar = tarfile.open('/mnt/data/news20.tar.gz', "r:gz")
tar.extractall(path="/content/news20")
tar.close()

## 4. Функція для обробки тексту

In [50]:
import re

def preprocess_text(text):
    # Видаляємо службову інформацію
    text = re.sub(r'(Path:|Newsgroups:|writes:|Subject:|From:).*', '', text)
    # Видаляємо непотрібні символи і залишаємо тільки латинські літери та апострофи
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    # Перетворюємо в нижній регістр
    text = text.lower()
    return text

## 5. Зчитування даних і створення RDD

In [51]:
import os

data_path = "/content/news20/20_newsgroup"

# Створюємо RDD з документів
documents = []
for root, dirs, files in os.walk(data_path):  # Проходимо по всіх файлах в директорії
    for file in files:
        file_path = os.path.join(root, file)
        with open(file_path, 'r', encoding='latin-1') as f:
            content = f.read()
            preprocessed_content = preprocess_text(content)
            documents.append((file, preprocessed_content))  # Додаємо кортеж (ім'я файлу, оброблений текст) до списку

rdd = sc.parallelize(documents)  # Створюємо RDD з документів

## 6. Створення інвертованого індексу

In [52]:
def create_inverted_index(doc_id, content):
    words = content.split()  # Розбиваємо текст на слова
    return [(word, doc_id) for word in set(words)]  # Повертаємо список пар (слово, ідентифікатор документа)

inverted_rdd = rdd.flatMap(lambda x: create_inverted_index(x[0], x[1]))  # Створюємо RDD з парами (слово, документ)

## 7. Підрахунок входжень та формування індексу

In [53]:
# Підрахунок загальної кількості входжень та створення списку документів
inverted_index = (
    inverted_rdd
    .map(lambda x: (x[0], [x[1]]))  # Створюємо пари (слово, [документ])
    .reduceByKey(lambda a, b: a + b)  # Об'єднуємо всі списки документів для одного слова
    .map(lambda x: (x[0], len(x[1]), ' '.join(sorted(set(x[1])))))  # Формуємо кортеж (слово, кількість документів, список документів)
)

## 8. Видаляємо попередній CSV файл, якщо він існує

In [54]:
!rm -rf /content/index.csv

## 9. Збереження та перевірка результату

In [55]:
# Конвертуємо результат у DataFrame для збереження у CSV
inverted_df = inverted_index.toDF(["word", "total_count", "documents"])

# Зберігаємо результат у CSV
inverted_df.coalesce(1).write.csv("/content/index.csv", header=True)

# Відображаємо результат
inverted_df.show(truncate=False)

+--------+-----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------